In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1"
os.environ['KERAS_BACKEND'] = "tensorflow"
import keras
import keras.layers as lay
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from src.generator import Generator
from src.discriminator import Discriminator
from src.aux_fn import train_step

2025-09-03 18:34:31.804373: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756924471.837930 3107576 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756924471.848421 3107576 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756924471.874056 3107576 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756924471.874094 3107576 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756924471.874098 3107576 computation_placer.cc:177] computation placer alr

In [2]:
(trainx,trainy),(_,_) = keras.datasets.cifar10.load_data()
trainx = tf.cast(trainx[trainy[:,0]==1],tf.float32)/255.
train4 = tf.image.resize(trainx,(4,4))
data = tf.data.Dataset.from_tensor_slices((train4)).cache().shuffle(2**10).batch(2**8).prefetch(tf.data.AUTOTUNE)

I0000 00:00:1756924479.498563 3107576 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10155 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:04:00.0, compute capability: 8.6


In [3]:
gen = Generator(name='Generator')
dis = Discriminator(name='Discriminator')
gen.build()
dis.build()
gen.summary()
dis.summary()

Model: "Generator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ toRGB4 (Conv2D)                 │ (None, 4, 4, 3)        │           387 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ toRGB8 (Conv2D)                 │ (None, 8, 8, 3)        │           387 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ toRGB16 (Conv2D)                │ (None, 16, 16, 3)      │           387 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ toRGB32 (Conv2D)                │ (None, 32, 32, 3)      │           387 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv0 (Conv2D)           │ (None, 4, 4, 128)      │       589,952 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 4, 4, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block8_conv0 (Conv2D)           │ (None, 8, 8, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block8_conv1 (Conv2D)           │ (None, 8, 8, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block16_conv0 (Conv2D)          │ (None, 16, 16, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block16_conv1 (Conv2D)          │ (None, 16, 16, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block32_conv0 (Conv2D)          │ (None, 32, 32, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block32_conv1 (Conv2D)          │ (None, 32, 32, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ upsample8 (UpSampling2D)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ upsample16 (UpSampling2D)       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ upsample32 (UpSampling2D)       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fade_in4_8 (Fade_in)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fade_in8_16 (Fade_in)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fade_in16_32 (Fade_in)          │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Brain (Dense)                   │ (None, 8192)           │     1,056,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Reshape (Reshape)               │ (None, 4, 4, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,681,356 (10.23 MB)

 Trainable params: 2,681,356 (10.23 MB)

 Non-trainable params: 0 (0.00 B)

Model: "Discriminator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ downsample4_0                   │ ?                      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ downsample4_1                   │ ?                      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ downsample8_0                   │ ?                      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ downsample8_1                   │ ?                      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ downsample16_0                  │ ?                      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ downsample16_1                  │ ?                      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Brain (Dense)                   │ (None, 1)              │         2,049 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fade_in_4_8 (Fade_in)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fade_in_8_16 (Fade_in)          │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fade_in_16_32 (Fade_in)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fromRGB4                        │ (None, 4, 4, 128)      │         3,712 │
│ (SpectralNormalization)         │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fromRGB8                        │ (None, 8, 8, 128)      │         3,712 │
│ (SpectralNormalization)         │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fromRGB16                       │ (None, 16, 16, 128)    │         3,712 │
│ (SpectralNormalization)         │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fromRGB32                       │ (None, 32, 32, 128)    │         3,712 │
│ (SpectralNormalization)         │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv0                    │ (None, 4, 4, 128)      │       147,712 │
│ (SpectralNormalization)         │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1                    │ (None, 4, 4, 128)      │       147,712 │
│ (SpectralNormalization)         │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block8_conv0                    │ (None, 8, 8, 128)      │       147,712 │
│ (SpectralNormalization)         │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block8_conv1                    │ (None, 8, 8, 128)      │       147,712 │
│ (SpectralNormalization)         │                        │             

 Total params: 1,198,593 (4.57 MB)

 Trainable params: 1,197,057 (4.57 MB)

 Non-trainable params: 1,536 (6.00 KB)

In [4]:
bce = keras.losses.BinaryCrossentropy()
opt = [keras.optimizers.Adam(1e-4,.5) for i in range(2)]

In [5]:
for i in range(10):
    for batch in data:
        gl, dl = train_step([gen,dis],batch,bce,opt,stage=0)
    print(f'Gloss = {gl:.4f}; Dloss = {dl:.4f}:')

2025-09-03 18:34:47.876939: W tensorflow/core/kernels/data/cache_dataset_ops.cc:916] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


ValueError: in user code:

    File "/home/villeneve/ProgressiveGAN/src/aux_fn.py", line 25, in train_step  *
        true_logits = dis(batch,training=True)
    File "/home/villeneve/venv311/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py", line 122, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/home/villeneve/ProgressiveGAN/src/discriminator.py", line 148, in call
        return tf.switch_case(self.stage.read_value(),branch_fn)
    File "/home/villeneve/ProgressiveGAN/src/discriminator.py", line 143, in <lambda>
        lambda: forward_8x8(inputs),
    File "/home/villeneve/ProgressiveGAN/src/discriminator.py", line 95, in forward_8x8
        x = self.brain(x)

    ValueError: Exception encountered when calling Discriminator.call().
    
    [1mInput 0 of layer "Brain" is incompatible with the layer: expected axis -1 of input shape to have value 2048, but received input with shape (256, 512)[0m
    
    Arguments received by Di

In [ ]:
img = gen(tf.random.normal((1,128)))
plt.imshow(np.uint8(img[0]*127.5+127.5))
plt.show()